In [ ]:
!pip install ultralytics -q

In [ ]:
from ultralytics import YOLO
import os
import shutil
import cv2
import matplotlib.pyplot as plt
import yaml
import numpy as np
import cv2
from matplotlib import pyplot as plt
import math
import torch

In [ ]:
src = "/kaggle/input/licenseplates"
dst = "/kaggle/working/licenseplates"

# Copy toàn bộ dataset sang working
if os.path.exists(dst):
    shutil.rmtree(dst)  # xóa nếu đã tồn tại để tránh lỗi

shutil.copytree(src, dst)
print("Đã copy dataset sang:", dst)

# Tạo file dataset.yaml với đường dẫn mới
dataset_yaml = {
    'train': os.path.join(dst, 'images/train'),
    'val': os.path.join(dst, 'images/val'),
    'nc': 2,
    'names': ['BSD', 'BSV']
}

yaml_path = os.path.join(dst, 'dataset.yaml')
with open(yaml_path, 'w') as f:
    yaml.dump(dataset_yaml, f, default_flow_style=False)

print("Đã tạo dataset.yaml tại:", yaml_path)

# Kiểm tra nội dung yaml
with open(yaml_path) as f:
    print(f.read())

In [ ]:
# Khởi tạo model segmentation YOLOv8
model = YOLO("yolov8s-seg.pt")

# Train
results = model.train(
    data="/kaggle/working/licenseplates/dataset.yaml",  # đường dẫn tới file yaml
    epochs=50,          # số epoch (tùy chỉnh)
    imgsz=640,          # resize ảnh
    batch=-1,           # auto batch size
    device=0,            # dùng GPU
    project="models/yolo", # thư mục cha để chứa kết quả
    name="yolov8s_licenseplates_det" # tên cụ thể cho lần chạy (subfolder trong project)
)

In [ ]:
# Test model
def detect_plate(yolo_model_path, img_path, crop_by="bbox"):
    # Load model
    model = YOLO(yolo_model_path)

    # Đọc ảnh
    img = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Chạy YOLO
    results = model(img_rgb)

    # Lấy kết quả box
    bboxes = results[0].boxes.xyxy.cpu().numpy()   # (x1,y1,x2,y2)
    scores = results[0].boxes.conf.cpu().numpy()
    class_ids = results[0].boxes.cls.cpu().numpy()

    # Lấy mask polygon (nếu có)
    masks = results[0].masks.xy if results[0].masks is not None else None

    print("Số đối tượng phát hiện:", len(scores))

    plate_crops = []  # list lưu các ảnh cắt

    for i, (bbox, score, class_id) in enumerate(zip(bboxes, scores, class_ids)):
        x1, y1, x2, y2 = map(int, bbox)

        if crop_by == "bbox" or masks is None:
            # ---- Crop theo bounding box ----
            crop = img_rgb[y1:y2, x1:x2]

        elif crop_by == "poly":
            # ---- Crop theo polygon mask ----
            polygon = masks[i].astype(np.int32)  # polygon: [[x1,y1],[x2,y2],...]
            mask = np.zeros(img_rgb.shape[:2], dtype=np.uint8)
            cv2.fillPoly(mask, [polygon], 255)

            # Áp dụng mask để lấy vùng polygon
            crop_masked = cv2.bitwise_and(img_rgb, img_rgb, mask=mask)

            # Crop theo bounding box nhỏ nhất chứa polygon
            x, y, w, h = cv2.boundingRect(polygon)
            crop = crop_masked[y:y+h, x:x+w]

        else:
            raise ValueError("crop_by phải là 'bbox' hoặc 'poly'")

        plate_crops.append(crop)

        # Vẽ box lên ảnh gốc
        cv2.rectangle(img_rgb, (x1, y1), (x2, y2), (0, 255, 0), 2)
        label = f'Class {int(class_id)}: {score:.2f}'
        cv2.putText(img_rgb, label, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)

    # Hiển thị ảnh có box
    plt.axis('off')
    plt.imshow(img_rgb)
    plt.show()

    return plate_crops